In [ ]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
from SimCLR_data_util import preprocess_for_train
from resnet_small import ResNet18
from tensorflow.keras.layers import Dense
from viewmaker import *
from logistic_regression import *

In [5]:
from datasets.cifar_10 import get_unsupervised_dataset
dataset = get_unsupervised_dataset(batch_size=64) 

InternalError: Failed copying input tensor from /job:localhost/replica:0/task:0/device:CPU:0 to /job:localhost/replica:0/task:0/device:GPU:0 in order to run _EagerConst: Dst tensor is not initialized.

In [ ]:
augmentation = Viewmaker()

In [ ]:
def get_projection_head():

    projection_head = tf.keras.Sequential([
        Dense(1024, activation='relu'),
        tf.keras.layers.Dropout(0.5),
        Dense(512, activation='relu'),
        tf.keras.layers.Dropout(0.5),
        Dense(256)
    ])

    return projection_head

projection_head = get_projection_head()

In [ ]:
def get_encoder():
    model = ResNet18(10)
    encoder = tf.keras.Sequential(model.layers[:-1])
    return encoder

encoder = get_encoder()

In [ ]:
viewmaker_model = SimCLR(encoder, augmentation, projection_head, temperature=0.07)

In [ ]:
viewmaker_model.load_weights('./cifar_10_experiment/viewmaker_and_simclr_weights/ckpt1')
encoder = viewmaker_model.encoder

In [ ]:
from datasets.cifar_10 import get_supervised_dataset
dataset = get_supervised_dataset()     

In [ ]:
classifier = TrainClassifier(10, encoder)

In [ ]:
classifier.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.01), 
                    loss=tf.keras.losses.categorical_crossentropy,
                    metrics=[tf.keras.metrics.CategoricalAccuracy()])
x = tf.convert_to_tensor(dataset[0][0])
y = dataset[0][1]
classifier.fit(x=x, y=y, batch_size=512, epochs=100)